In [1]:
import requests
import urllib.parse as ulp
import mysql.connector

In [2]:
mydb = mysql.connector.connect(
host="web5.ces.census.gov",
user="##",
password="##",
database="pmt"
)
mycursor = mydb.cursor()

sql = "SET collation_connection = 'utf8_general_ci'"
mycursor.execute(sql)
mydb.commit()
sql1 = "ALTER TABLE api_core CONVERT TO CHARACTER SET utf8 COLLATE utf8_general_ci"
mycursor.execute(sql1)
mydb.commit()

In [3]:
#https://core.ac.uk:443/api-v2/search/"Ronald Jarmin"?page=1&pageSize=20&apiKey=##
class Core:
        
    def sendToDB(self, proj_id, results, hits):
        #print(proj_id)
        mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user="##",
        password="##",
        database="pmt"
        )

        mycursor = mydb.cursor()
        
        numPrint = 125
        if hits == None:
            numPrint = 0
        elif hits < 125:
            numPrint = hits
        #print("numPrint = " + str(numPrint))
        for i in range(numPrint):
            title = str(results[1]['data'][i]['_source']['title'])
            if results[1]['data'][i]['_source']['doi'] is not None:
                doi = results[1]['data'][i]['_source']['doi']
            else:
                doi = "None"
            #work_id = str(results[1]['data'][i]['_id'])
            author = results[1]['data'][i]['_source']['authors']
            if results[1]['data'][i]['_source']['citations'] is not None:
                citations = str(results[1]['data'][i]['_source']['citations'])
            else:
                citations = "None"
            contributors = results[1]['data'][i]['_source']['contributors']
            if results[1]['data'][i]['_source']['datePublished'] is not None:
                pub_year = str(results[1]['data'][i]['_source']['datePublished'])[:4]
                pub_month = str(results[1]['data'][i]['_source']['datePublished'])[5:7]
            else:
                pub_year = "None"
                pub_month = "None"
            if results[1]['data'][i]['_source']['topics'] is not None:
                topics = str(results[1]['data'][i]['_source']['topics'])
            else:
                topics = "None"
            full_text = str(results[1]['data'][i]['_source']['fullText'])
            full_text_link = str(results[1]['data'][i]['_source']['downloadUrl'])
            publisher = str(results[1]['data'][i]['_source']['publisher'])
            abstract = str(results[1]['data'][i]['_source']['description'])
            doc_id = str(results[1]['data'][i]['_id'])
            
            sql_dup = "SELECT * FROM api_core WHERE doc_id = %s AND proj_id = %s"
            data = (doc_id, proj_id)
            mycursor.execute(sql_dup, data)
            duplicates = mycursor.fetchall()
            #print(len(duplicates))
            #for dup in duplicates:
                #print(dup)
            if len(duplicates) == 0:
                sql = "INSERT INTO api_core (proj_id, title, doi, pub_year, pub_month, topics, citations, abstract, full_text, full_text_link, publisher, doc_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                val = (proj_id, title, doi, pub_year, pub_month, topics, citations, abstract, full_text, full_text_link, publisher, doc_id)
            
            #print("new val")
            #print(val)
            
                mycursor.execute(sql, val)
                pub_id = mycursor.lastrowid
                print("pub_id: " +str(pub_id))

                mydb.commit()

                print(mycursor.rowcount, "record inserted.")

                if bool(author):
                    for j in range(len(author)):
                        sql_part = "INSERT INTO api_core_participants (pub_id, proj_id, fullname) VALUES (%s, %s, %s)"
                        val_part = (pub_id, proj_id, author[j])

                        mycursor.execute(sql_part, val_part)

                        mydb.commit()

                        print(mycursor.rowcount, "participant record inserted.")
                
                if bool(contributors):
                    for j in range(len(contributors)):
                        sql_cont = "INSERT INTO api_core_participants (pub_id, proj_id, fullname) VALUES (%s, %s, %s)"
                        val_cont = (pub_id, proj_id, contributors[j])

                        mycursor.execute(sql_cont, val_cont)

                        mydb.commit()

                        print(mycursor.rowcount, "participant record inserted.")

            else:
                print("duplicate value, not added")
        
        sql_update_doi = "UPDATE api_core SET doi=NULL WHERE doi='None'"
        mycursor.execute(sql_update_doi)
        mydb.commit()
        sql_update_title = "UPDATE api_core SET title=NULL WHERE title='None'"
        mycursor.execute(sql_update_title)
        mydb.commit()
        sql_update_year = "UPDATE api_core SET pub_year=NULL WHERE pub_year='None'"
        mycursor.execute(sql_update_year)
        mydb.commit()
        sql_update_month = "UPDATE api_core SET pub_month=NULL WHERE pub_month='None'"
        mycursor.execute(sql_update_month)
        mydb.commit()
        sql_update_link = "UPDATE api_core SET full_text_link=NULL WHERE full_text_link='None'"
        mycursor.execute(sql_update_link)
        mydb.commit()
        sql_update_publisher = "UPDATE api_core SET publisher=NULL WHERE publisher='None'"
        mycursor.execute(sql_update_publisher)
        mydb.commit()
        sql_update_abstract = "UPDATE api_core SET abstract=NULL WHERE abstract='None'"
        mycursor.execute(sql_update_abstract)
        mydb.commit()
        sql_update_id = "UPDATE api_core SET doc_id=NULL WHERE doc_id='None'"
        mycursor.execute(sql_update_id)
        mydb.commit()
        print("changed None values to null in doi, title, year, month, full text link, publisher, abstract, doc id")
    
class CoreSearch(Core):
        
    def search(self, _path, proj_id, _query):
        header = ["Accept: application/json", "apiKey=##"]
        url = "https://core.ac.uk:443/api-v2"
        #print(url+_path+ulp.quote(_query)+"?page=1&pageSize=20&"+header[1])
        request = requests.get(url+_path+_query+"?page=1&pageSize=125&"+header[1], proxies={'https':'##'})
        
        if (request.status_code == 200):
            data = [proj_id, request.json()]
            return data
    
    def searchAll(self, proj_id, _query):
        return self.search("/search/", proj_id, _query)
    
    def searchArticles(self, _query):
        return self.search("/articles/search/", _query)
        
    # search results return null?
    def searchJournals(self, _query):
        return self.search("/journals/search/", _query)   
    
    # search results return null?
    def searchRepositories(self, _query):
        return self.search("/repositories/search/", _query)

In [4]:
# mydb = mysql.connector.connect(
#         host="web5.ces.census.gov",
#         user="##",
#         password="##",
#         database="pmt"
#         )

# mycursor = mydb.cursor()

# test = "ALTER TABLE api_core AUTO_INCREMENT = 1"
# mycursor.execute(test)
# mydb.commit()
# print("altered auto-increment")

altered auto-increment


In [4]:
from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

#https://www.ces.census.gov/api/iaa/v1.3/project/status/CP?fields=cms_project_id,title&page=0&limit=10
def run_query(path, fields, page, limit):
    request = requests.get('https://www.ces.census.gov/api/iaa/v1.3%s?fields=%s&page=%s&limit=%s' % (path,fields,page,limit), headers=headers, proxies={"https" : "148.129.75.18:3128"})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}.".format(request.status_code))

def name_and_ds(proj_id): 
    data = {}
    proj = run_query('/project/'+str(proj_id),'cms_project_id,title','0','10') # Execute the project query
    #'/project/status/CP'
    full = proj[0]['title']
    parts = full.split("-")
    data['title']= parts[2][1:]
    #print(proj['cms_project_id'])
    datasets = run_query('/project/%s/data' % proj[0]['cms_project_id'],'name','0','10') # Execute the dataset query
    sets = []

    for ds in datasets:
        sets.append(ds['name'])

    data['datasets']=sets
    return data


In [5]:
import base64
from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

def run__abstract_query(path,fields,page,limit):
    request = requests.get('https://www.ces.census.gov/api/iaa/v1.3%s?fields=%s&page=%s&limit=%s' % (path,fields,page,limit), headers=headers, proxies={"https" : "148.129.75.18:3128"})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}.".format(request.status_code))
        
def get_abstract(projid):
    results = run__abstract_query('/project/'+str(projid)+'/proposal/text','mime_data','0','10')
    #print(results)
    return results[0]['mime_data'].replace("\n", " ")

In [6]:
'''cms authors'''
import base64
from base64 import b64encode
from pprint import pprint
import requests

api_key = ##

headers = { 'Authorization' : 'key=%s' %  (api_key).decode("ascii") }

def get_authors(pjid):
    projects = run_query('/project/'+str(pjid),'cms_project_id,type_name,title,participants{username,fullname,roles}','0','100') # Execute the query

    auts = []
    
    for proj in projects:
        #print(proj['title'])
        if proj['title'].split("-")[0] == str(pjid) + " ":
            for part in proj['participants']:
                if 'researcher' in part['roles']:
                    auts.append(part['fullname'])
                    
    return auts

In [7]:
def auto_search(pjid, author):
    queries = []
    queries.append(name_and_ds(pjid).get('title'))

    part = author.partition(' ')
    aut1 = author
    aut2 = part[0] + ' ' + part[2].partition(' ')[2]
    first = part[0]
    middle = part[2].partition(' ')[0]
    last = part[2].partition(' ')[2]
    #print(part[2].partition(' ')[2])
    if len(part[0]) == 1:
        first = part[0]+"."
    if len(part[2].partition(' ')[0]) == 1:
        middle = part[2].partition(' ')[0] + "."
    if len(part[2].partition(' ')[2]) == 1:
        last = part[2].partition(' ')[2] + "."
    aut3 = first + " " + last
    aut4 = first + " " + middle + " " + last
    authors = [aut1, aut2, aut3, aut4]
    for i in range(len(authors)):
        if authors[i] not in queries:
            queries.append(authors[i])
    #print(queries)
    
    api = CoreSearch()
    for i in range(len(queries)):
        results = api.searchAll(pjid, '"'+queries[i]+'"')
        print("'"+queries[i]+"'")
        print("Total hits: " + str(results[1]['totalHits']))
        #print("proj_id: " +str(results[0]))
        hits = results[1]['totalHits']
        #proj_id = str(results[0])
        api.sendToDB(pjid, results, hits)

In [19]:
auto_search(1745, "Hongwei Xu")

'Improving Estimates for Post'
Total hits: 0
changed None values to null in doi, title, year, month, full text link, publisher, abstract, doc id
'Hongwei Xu'
Total hits: 512
pub_id: 1813
1 record inserted.
1 participant record inserted.
1 participant record inserted.
pub_id: 1814
1 record inserted.
1 participant record inserted.
1 participant record inserted.
pub_id: 1815
1 record inserted.
1 participant record inserted.
1 participant record inserted.
1 participant record inserted.
pub_id: 1816
1 record inserted.
1 participant record inserted.
1 participant record inserted.
1 participant record inserted.
pub_id: 1817
1 record inserted.
1 participant record inserted.
1 participant record inserted.
1 participant record inserted.
pub_id: 1818
1 record inserted.
1 participant record inserted.
1 participant record inserted.
1 participant record inserted.
1 participant record inserted.
pub_id: 1819
1 record inserted.
1 participant record inserted.
1 participant record inserted.
pub_id: 1820


In [8]:
import nltk
#nltk.set_proxy('http://proxy.##')
#nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import FreqDist

In [22]:
def search_NLP(pjid, year):
    
    mydb = mysql.connector.connect(
        host="web5.ces.census.gov",
        user="##",
        password="##",
        database="pmt"
        )

    mycursor = mydb.cursor()
    
    import nltk
    #nltk.set_proxy('http://proxy.tco##')
    #nltk.download("punkt")
    from nltk.corpus import stopwords
    from nltk.tokenize import sent_tokenize, word_tokenize
    from nltk import FreqDist
    from nltk import ngrams

    '''NLP'''
    title = name_and_ds(pjid)['title']
    description = title+' '+get_abstract(pjid)

    #tokenizing words and getting rid of stop words
    descrip = word_tokenize(description)

    stop_words = set(stopwords.words("english"))

    filtered_descrip = []
    filtered_list = [
        word for word in descrip if word.casefold() not in stop_words]
    #print(filtered_list)

    #tagging POS
    descrip_pos = nltk.pos_tag(descrip)

    #chunking-- phrases with a specific format
    #noun phrase that starts w/any # adjs, and ends w/any # nouns
    grammar = "NP: {<JJ>*<NN>*}"
    chunk_parser = nltk.RegexpParser(grammar)
    tree = chunk_parser.parse(descrip_pos)
    phrase = []
    # stores all the phrases, key indexed by the # of words in the phrase
    chunks = {}
    for s in tree.subtrees(filter=lambda tree: tree.label() == 'NP'):
        #print([x[0] for x in s.leaves()])
        for x in s.leaves():
            phrase.append(x[0])
        #print(phrase)
        size = len(phrase)
        full = ""
        for i in range(size):
            if full=="":
                full = phrase[i]
            else:
                full = full+" "+phrase[i]
        if chunks.get(str(size)) is not None:
            chunks.get(str(size)).append(full)
        else:
            chunks[str(size)] = [full]
        phrase = []
    # print("chunks")
    # print(chunks)           
    #tree.draw()

    #freq distribution
    freq_words = []
    frequency_distribution = FreqDist(filtered_list)
    #print(frequency_distribution.most_common(20))
    for i in range(20):
        word = frequency_distribution.most_common(20)[i][0]
        if len(word) == 1:
            continue
        else:
            freq_words.append(word)
    # print("most frequent words")
    # print(freq_words)

    #print(collections.counter())

    #collocation- sequence of words that shows up often
    # print("collocations")
    # print(nltk.Text(description).collocation_list())

    #ngrams, groups of n words
    from nltk import ngrams, FreqDist
    all_ngrams = {}
    all_counts = dict()
    for size in 2, 3, 4, 5:
        all_counts[size] = FreqDist(ngrams(filtered_list, size))
    #print(all_counts[2].most_common(5))
    third=''
    fourth=''
    fifth=''
    for i in range(4):
        for j in range(5):
            first = all_counts[i+2].most_common(5)[j][0][0]
            second = all_counts[i+2].most_common(5)[j][0][1]
            if i+2 == 2:
                if len(first) == 1 or len(second) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams.get(str(i+2)).append(first + " " + second)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second]
            if i+2 == 3:
                third = all_counts[i+2].most_common(5)[j][0][2]
                if len(first) == 1 or len(second) == 1 or len(third) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams.get(str(i+2)).append(first + " " + second + " " + third)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second + " " + third]
            if i+2 == 4:
                third = all_counts[i+2].most_common(5)[j][0][2]
                fourth = all_counts[i+2].most_common(5)[j][0][3]
                if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth]
            if i+2 == 5:
                third = all_counts[i+2].most_common(5)[j][0][2]
                fourth = all_counts[i+2].most_common(5)[j][0][3]
                fifth = all_counts[i+2].most_common(5)[j][0][4]
                if len(first) == 1 or len(second) == 1 or len(third) == 1 or len(fourth) == 1 or len(fifth) == 1:
                    continue
                if all_ngrams.get(str(i+2)) is not None:
                    all_ngrams.get(str(i+2)).append(first + " " + second + " " + third + " " + fourth + " " + fifth)
                else:
                    all_ngrams[str(i+2)] = [first + " " + second + " " + third + " " + fourth + " " + fifth]

    #print("ngrams")                
    #print(all_ngrams)
    #print(all_counts[3].most_common(5))

    keywords = []
    if all_ngrams.get("2") is not None:
        keywords += all_ngrams.get("2")
    if chunks.get("2") is not None:
        keywords += chunks.get("2")
    if chunks.get("3") is not None:
        keywords += chunks.get("3")
    #print("keywords w/duplicates")
    #print(keywords)
    #print(len(keywords))

    #print("final keywords")
    kw_set = set(keywords) #caps sensitive-- same words but dif capitalization don't get removed
    keywords = list(kw_set)
    #print(keywords)
    #print(len(keywords))
    print("done NLP")

    '''end NLP'''

    #description = "The project will proceed in three phases. The first will enhance the export and import information on Economic Censuses and Surveys. We will develop and test linkages between transaction level Foreign Trade data and Economic Census and Survey data and compare the links developed by CES to those developed by the Census Bureau’s Foreign Trade Division (FTD). If improvements in linkages methods are identified, we will provide advice to FTD. The second phase will analyze transaction level detail to examine changes in foreign trade transactions, focusing initially on related party transactions, potential underreporting, and impact of FTD outreach efforts. This component of the project will focus on identifying reasons for such changes. The third phase of the project will develop empirical and analytical framework to investigate the impact of trade on the U.S. economy - focusing on how firms allocate economic activity between domestic and foreign production and the impact of this on the domestic economy (including workers and regional economies). This component makes use of the data developed in the previous phases to develop new estimates of the impact of foreign trade on U.S. industries. It will examine how imports and exports affect domestic production, employment, and productivity. It will also examine how firm responses to trade affect local labor market outcomes."
    # key is author's name, values are the queries for that name
    # for each author name present, get an additional point

    '''inputs'''
    #pjid = 712 #given
    #year = 2009 #given

    res = {} # of points per pub_id for each pjid
    matches = {} # of hits for each query for each pjid
    authors = {}

    auts = get_authors(pjid) #array of authors' names
    numAuts = len(auts)
    title = name_and_ds(pjid)['title']
    # for each author, split into two search queries-- one w/ and one w/out middle initial

    for i in range(numAuts):
        part = auts[i].partition(' ')
        aut1 = auts[i]
        aut2 = part[0] + ' ' + part[2].partition(' ')[2]
        first = part[0]
        middle = part[2].partition(' ')[0]
        last = part[2].partition(' ')[2]
        #print(part[2].partition(' ')[2])
        if len(part[0]) == 1:
            first = part[0]+"."
        if len(part[2].partition(' ')[0]) == 1:
            middle = part[2].partition(' ')[0] + "."
        if len(part[2].partition(' ')[2]) == 1:
            last = part[2].partition(' ')[2] + "."
        aut3 = first + " " + last
        aut4 = first + " " + middle + " " + last
        authors[i] = [aut1, aut2, aut3, aut4]

    ds = name_and_ds(pjid)['datasets'] #array

    params_inner = {}
    params = {}

    params_inner['year'] = year
    params_inner['authors'] = auts
    params_inner['datasets'] = ds
    params_inner['keywords'] = keywords
    params[pjid] = params_inner

    #array of how many hits each query gets
    data = []

    '''pub_year'''
    q1 = "SELECT * FROM api_core WHERE proj_id= %s AND (pub_year >= %s OR pub_year is NULL)"
    d1 = (pjid, year)
    mycursor.execute(q1, d1)
    r1 = mycursor.fetchall()
    data.append(len(r1))
    rows = {}
    for i in range(len(r1)):
#         if rows.get(r1[i][0]) is not None:
#             rows[r1[i][0]] = rows.get(r1[i][0])+1
#         else:
        rows[r1[i][0]] = 1
    #print(r1)
    #print(type(rows))
    #print(rows)

    '''authors'''
    aut_hits = 0

    def refineAut(pjid, aut1, aut2, aut3, aut4):
        q2 = "SELECT * FROM api_core_participants WHERE proj_id= %s AND (fullname = %s OR fullname = %s OR fullname = %s OR fullname = %s)"
        d2 = (pjid, aut1, aut2, aut3, aut4)
        mycursor.execute(q2, d2)
        r2 = mycursor.fetchall()
        #aut_hits += len(r2)
        for i in range(len(r2)):
            if rows.get(r2[i][0]) is not None:
                rows[r2[i][0]] = rows.get(r2[i][0])+1
        return len(r2)

    for i in range(numAuts):
        aut_hits += refineAut(pjid, authors.get(i)[0], authors.get(i)[1], authors.get(i)[2], authors.get(i)[3])
        #print(authors.get(i))
        #print(aut_hits)

    data.append(aut_hits)
    
    for key in rows.keys():
        if rows.get(key) == 1:
            rows[key] = 0.5

    nlp_matches = {}
    nlp_data = []
    nlp_rows = {}
    nlp_res = {}

    '''queries 3 and 4: one pt each for having at least one matching word, 1 add. pt for each matching word'''
    '''title keywords in description'''
    q3 = "SELECT * FROM api_core WHERE proj_id= %s AND (abstract LIKE %s"
    d3 = (pjid, keywords[0])
    l3 = list(d3)
    numData = len(keywords)
    for i in range(numData-1):
        q3 += " OR abstract LIKE %s"
        if i == numData-2:
            q3 += ")"
        l3.append(str('%' + keywords[i+1] + '%'))
    if numData == 1:
        q3 += ")"
    d3 = tuple(l3)
    mycursor.execute(q3, d3)
    r3 = mycursor.fetchall()
    nlp_data.append(len(r3)) #amount of hits that have at least one matching word
    for i in range(len(r3)):
        if nlp_rows.get(r3[i][0]) is not None:
            nlp_rows[r3[i][0]] = nlp_rows.get(r3[i][0])+1
        else:
            nlp_rows[r3[i][0]] = 1

    '''title keywords in title'''
    q4 = "SELECT * FROM api_core WHERE proj_id= %s AND (title LIKE %s"
    d4 = (pjid, keywords[0])
    l4 = list(d4)
    numData = len(keywords)
    for i in range(numData-1):
        q4 += " OR title LIKE %s"
        if i == numData-2:
            q4 += ")"
        l4.append(str('%' + keywords[i+1] + '%'))
    if numData == 1:
        q4 += ")"
    d4 = tuple(l4)
    mycursor.execute(q4, d4)
    r4 = mycursor.fetchall()
    nlp_data.append(len(r4)) #amount of hits with at least one matching word
    for i in range(len(r4)):
        if nlp_rows.get(r4[i][0]) is not None:
            nlp_rows[r4[i][0]] = nlp_rows.get(r4[i][0])+1
        else:
            nlp_rows[r4[i][0]] = 1

    descrips = {}
    #grab and store titles and descriptions of all publications that had at least one match in title or descrip
    for key in nlp_rows.keys():
        #print(type(key))
        q = "SELECT title FROM api_core WHERE pub_id = " +str(key)
        mycursor.execute(q)
        r = mycursor.fetchall()
        descrips[key] = [r[0][0]]
        qq = "SELECT abstract FROM api_core WHERE pub_id = " +str(key)
        mycursor.execute(qq)
        rr = mycursor.fetchall()
        descrips.get(key).append(rr[0][0])

    #print(descrips)
    #print(nlp_rows)

    title_hits = 0
    descrip_hits = 0

    #print(keywords[0])
    #if descrips.keys() is not None:
    for i in range(len(keywords)):
        for key in descrips.keys():
            #print(key)
            #(keywords[i])
            #print(descrips[key][0])
            #print(key)
            #print(nlp_rows)
            if descrips[key][0] is not None:
                if keywords[i] in descrips[key][0]:
                    if nlp_rows[key] is not None:
                        nlp_rows[key] = nlp_rows.get(key)+1
                        title_hits += 1
            if descrips[key][1] is not None:
                if keywords[i] in descrips[key][1]:
                    if nlp_rows[key] is not None:
                        nlp_rows[key] = nlp_rows.get(key)+1
                        descrip_hits += 1

    data.append(title_hits)
    data.append(descrip_hits)

    #print(nlp_rows)
    nlp_matches[pjid] = nlp_data
    #print(nlp_matches)
    nlp_res[pjid] = nlp_rows


    '''datasets in description'''
    q5 = "SELECT * FROM api_core WHERE proj_id= %s AND (abstract LIKE %s"
    d5 = (pjid, ds[0])
    l5 = list(d5)
    numData = len(ds)
    for i in range(numData-1):
        q5 += " OR abstract LIKE %s"
        if i == numData-2:
            q5 += ")"
        l5.append(str('%' + ds[i+1] + '%'))
    if numData == 1:
        q5 += ")"
    d5 = tuple(l5)
    #print(q5)
    #print(d5)
    mycursor.execute(q5, d5)
    r5 = mycursor.fetchall()
    data.append(len(r5))
    for i in range(len(r5)):
        if rows.get(r5[i][0]) is not None:
            rows[r5[i][0]] = rows.get(r5[i][0])+1
        #else:
            #rows[r5[i][0]] = 1
    #print("r5:")
    #print(r5)
    #print(rows)

    for key in rows.keys():
        if nlp_rows.get(key) is not None:
            rows[key] = rows.get(key)+nlp_rows.get(key)

    #print(rows)

    res_sorted = {}

    for w in sorted(rows, key = rows.get, \
                    reverse = True):
        res_sorted[w] = rows[w]
        #print(w, rows[w])

    print("sorted results:")
    print(res_sorted)
    #print(rows)
    res[pjid] = rows
    matches[pjid] = data
    #print(res)
    print("matches:")
    print(matches)
    #print("params:")
    #print(params)
    #print("total hits: " + str(counter))

In [25]:
search_NLP(1745,2018)

done NLP
sorted results:
{1989: 2.5, 1837: 2, 1843: 2, 1853: 2, 1905: 2, 1978: 2, 2049: 2, 1958: 1.5, 1815: 0.5, 1816: 0.5, 1825: 0.5, 1870: 0.5, 1938: 0.5, 1939: 0.5, 1941: 0.5, 1942: 0.5, 1955: 0.5, 1956: 0.5, 1959: 0.5, 1960: 0.5, 1962: 0.5, 1963: 0.5, 1964: 0.5, 1971: 0.5, 1973: 0.5, 1986: 0.5, 1988: 0.5, 1990: 0.5, 1999: 0.5, 2005: 0.5, 2011: 0.5, 2027: 0.5, 2028: 0.5, 2032: 0.5, 2045: 0.5, 2047: 0.5, 2048: 0.5, 2050: 0.5, 2052: 0.5}
matches:
{1745: [39, 105, 0, 11, 0]}
